In [ ]:
#Fazendo a leitura dos dados
import random
random.seed(42) # define the seed (important to reproduce the results)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

data_set_path = 'C:\\Users\\mlsob\\Developer\\Notebooks\\credit-risk-predict\\dataset\\credit-risk-classification\\'

train = pd.read_csv(data_set_path+'train.csv', header=(0))
test = pd.read_csv(data_set_path+'test.csv', header=(0))

print("Número de linhas e colunas no conjunto de treinamento:", train.shape)
print("Número de linhas e colunas no conjunto de teste:", test.shape)
attributes = list(train.columns)
train.head(10)

In [ ]:
#Exclusão de atributo não relevante "Loan_ID" nos conjuntos de treino e teste
train.drop(['Loan_ID'],axis = 1, inplace = True)
train.head(10)
#print(train.shape)

In [ ]:
Loan_ID_test = test['Loan_ID']
test.drop(['Loan_ID'],axis = 1, inplace = True)
test.head(10)

In [ ]:
#Transformação de atributos categóricos em numéricos com One-ho-enconding para o conjunto de treino 
#train.head(10)
#train = pd.get_dummies(train)
#train.head(10)

In [ ]:
#test = pd.get_dummies(test)
#test.head(10)

In [ ]:
#Identificação de valores nulos (not a number: nan), no conjunto de treino, com pandas
train.isnull().sum().sort_values(ascending=False).head(10)



In [ ]:
#Identificação de valores nulos (not a number: nan), no conjunto de teste, com pandas
test.isnull().sum().sort_values(ascending=False).head(10)

In [ ]:
#Substituição de valores faltantes em cada instância no conjuntos de treino e teste por valores válidos (moda, média etc)
train['Credit_History'] = train['Credit_History'].fillna(train['Credit_History'].mode()[0])
train['LoanAmount'] = train['LoanAmount'].fillna(train['LoanAmount'].mode()[0])
train['Loan_Amount_Term'] = train['Loan_Amount_Term'].fillna(train['Loan_Amount_Term'].mode()[0])

test['Credit_History'] = test['Credit_History'].fillna(test['Credit_History'].mode()[0])
test['LoanAmount'] = test['LoanAmount'].fillna(test['LoanAmount'].mode()[0])
test['Loan_Amount_Term'] = test['Loan_Amount_Term'].fillna(test['Loan_Amount_Term'].mode()[0])

In [ ]:
train.isnull().sum().sort_values(ascending=False).head(10)

In [ ]:
test.isnull().sum().sort_values(ascending=False).head(10)

In [ ]:
#converção dos dados para o formato numpy para facilitar tarefas de classificação e regressão
#Para o conjunto de treino
data_train = train.to_numpy()
print(data_train.shape)
nrow,ncol = data_train.shape
y = data_train[:,-1]
X = data_train[:,0:ncol-1]
print(y)
print(X)

In [ ]:
#Para o conjunto de teste
#data_test = test.to_numpy()
#X_test = data_test

In [ ]:
#Normalizados, de modo a evitar o efeito da escala dos atributos nos métodos de aprendizado
#from sklearn.preprocessing import StandardScaler
#scaler = StandardScaler().fit(X)
#X = scaler.transform(X)

#X_test = scaler.transform(X_test)

#print('Dados transformados:')
#print('Media: ', np.mean(X, axis = 0))
#print('Desvio Padrao:', np.std(X, axis = 0))

In [ ]:
from sklearn.model_selection import train_test_split
p = 0.2 # fracao de elementos no conjunto de teste
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = p, random_state = 2)

In [ ]:
from sklearn import tree
# Cria o modelo usando o criterio Gini
model = tree.DecisionTreeClassifier(criterion = 'gini', random_state = 101)
# Ajusta o modelo usando os dados de treinamento
model.fit(x_train,y_train)
# realizar a predição
y_pred = model.predict(x_test)